# 🚀 Deploy Policy with Observation Visualization

This notebook deploys a trained policy to a physical robot while visualizing the observation data (images and state) before sending each action.

### Features:
- **Live Image Display**: Shows camera observations in real-time
- **State Monitoring**: Displays robot state (joint positions, TCP poses, gripper states)
- **Action Preview**: Shows predicted actions before sending to robot
- **Debug-Friendly**: Great for understanding what the policy sees and predicts

### Process:
1. Configure deployment parameters
2. Load the trained policy
3. Set up visualization
4. Run deployment loop with live observation display

## 1. Configuration

First, specify the necessary parameters for deployment. **You must edit these values.**

In [ ]:
import pathlib

# TODO: Change to the directory containing your trained policy checkpoint.
# Example: "outputs/2025-09-14/12-00-00"
CHECKPOINT_DIR = pathlib.Path("[TODO]")

# TODO: Change to the robot's IP address.
SERVER_ENDPOINT = "localhost:50051"

# Inference frequency in Hz. Lower values give more time to display visualizations.
INFERENCE_FREQUENCY_HZ: float = 10.0

# Visualization settings
DISPLAY_IMAGES = True  # Set to False to only show state information
IMAGE_SCALE = 0.5  # Scale factor for displayed images (0.5 = 50% size)

print(f"Attempting to load policy from: {CHECKPOINT_DIR}")
print(f"Robot server endpoint: {SERVER_ENDPOINT}")
print(f"Inference frequency: {INFERENCE_FREQUENCY_HZ} Hz")
print(f"Display images: {DISPLAY_IMAGES}")

## 2. Load the Policy

Load the policy from the checkpoint directory.

In [ ]:
from example_policies.robot_deploy.deploy_core import policy_loader

policy, cfg = policy_loader.load_policy(CHECKPOINT_DIR)

print("✅ Policy loaded successfully!")

## 3. (Optional) Modify Policy Attributes

Adjust policy settings if needed.

In [ ]:
# Change the device on the config, not the policy!!
cfg.device = "cuda"
policy.to(cfg.device)  # or "cpu"

print(f"Device: {cfg.device}")

## 4. Setup Visualization Utilities

Import visualization libraries and create helper functions.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from PIL import Image

def display_observation(observation, action, step, clear=True):
    """Display observation images and state information."""
    if clear:
        clear_output(wait=True)
    
    # Extract image observations
    image_keys = [k for k in observation.keys() if k.startswith('observation.image')]
    
    if DISPLAY_IMAGES and image_keys:
        n_images = len(image_keys)
        fig, axes = plt.subplots(1, n_images, figsize=(5*n_images*IMAGE_SCALE, 4*IMAGE_SCALE))
        
        if n_images == 1:
            axes = [axes]
        
        for idx, img_key in enumerate(sorted(image_keys)):
            img_tensor = observation[img_key].squeeze(0)  # Remove batch dimension
            
            # Convert from CHW to HWC for display
            if img_tensor.shape[0] in [1, 3]:  # Channel first
                img_np = img_tensor.cpu().numpy().transpose(1, 2, 0)
            else:
                img_np = img_tensor.cpu().numpy()
            
            # Normalize if needed
            if img_np.max() <= 1.0:
                img_np = (img_np * 255).astype(np.uint8)
            
            axes[idx].imshow(img_np)
            axes[idx].set_title(img_key.replace('observation.image.', ''))
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Display state information
    print(f"\n{'='*60}")
    print(f"STEP {step}")
    print(f"{'='*60}")
    
    if 'observation.state' in observation:
        state = observation['observation.state'].squeeze(0).cpu().numpy()
        print(f"\n📊 STATE VECTOR (dim={len(state)}):")
        print(f"  Values: {state[:10]}..." if len(state) > 10 else f"  Values: {state}")
        print(f"  Min: {state.min():.4f}, Max: {state.max():.4f}, Mean: {state.mean():.4f}")
    
    # Display predicted action
    if action is not None:
        action_np = action.squeeze(0).cpu().numpy()
        print(f"\n🎯 PREDICTED ACTION (dim={len(action_np)}):")
        print(f"  Values: {action_np[:10]}..." if len(action_np) > 10 else f"  Values: {action_np}")
        print(f"  Min: {action_np.min():.4f}, Max: {action_np.max():.4f}")

print("✅ Visualization utilities ready!")

## 5. Deploy to Robot with Visualization

Execute the deployment loop with live observation display.

⚠️ **Warning**: This will move the physical robot. Ensure the robot has a clear and safe workspace.

The loop will:
1. Capture current observation (images + state)
2. Display the observation data
3. Run policy inference to predict action
4. **Wait for you to press ENTER**
5. Send action to robot
6. Repeat

Press Ctrl+C in the kernel to stop.

In [ ]:
import time
from example_policies.robot_deploy.deploy_core.deployment_structures import InferenceConfig
from example_policies.robot_deploy.deploy_core.policy_manager import PolicyManager
from example_policies.robot_deploy.robot_io.connection import RobotConnection
from example_policies.robot_deploy.robot_io.robot_interface import RobotClient, RobotInterface

# Load policy as a PolicyBundle
policy_bundle = PolicyManager.load_single(CHECKPOINT_DIR, cfg.device)

# Setup inference configuration
config = InferenceConfig(
    hz=INFERENCE_FREQUENCY_HZ,
    device=cfg.device,
    controller=RobotClient.CART_WAYPOINT,
)

# Run inference loop with visualization
print("Starting inference loop with visualization... Press Ctrl+C to stop.")
with RobotConnection(SERVER_ENDPOINT) as stub:
    robot_interface = RobotInterface(stub, policy_bundle.config)

    # Move robot to home position before starting inference
    print("Moving robot to home position...")
    robot_interface.move_home()
    
    step = 0
    try:
        while True:
            start_time = time.time()
            
            # Get observation
            observation = robot_interface.get_observation(policy_bundle.config.device)
            
            if not observation:
                time.sleep(1.0 / config.hz)
                continue
            
            # Run policy inference
            with torch.inference_mode():
                action = policy_bundle.policy.select_action(observation)
            
            # Translate action to robot commands
            action_translated = policy_bundle.translator.translate(action, observation)
            
            # Display observation and predicted action (without clearing)
            display_observation(observation, action, step, clear=False)
            
            # Print translated action info
            print(f"\n🤖 READY TO SEND TO ROBOT:")
            print(f"  Action mode: {policy_bundle.translator.action_mode}")
            print(f"  Controller: {config.controller}")
            
            # Wait for user confirmation
            user_input = input("\n⏸️  Press ENTER to send this action (or 'q' to quit)... ")
            
            if user_input.lower() == 'q':
                print("Stopping deployment...")
                break
            
            # Send action to robot
            robot_interface.send_action(
                action_translated,
                policy_bundle.translator.action_mode,
                config.controller,
            )
            
            print("✅ Action sent!")
            
            # Maintain frequency
            elapsed = time.time() - start_time
            sleep_time = (1.0 / config.hz) - elapsed
            if sleep_time > 0:
                time.sleep(sleep_time)
            
            step += 1
            
    except KeyboardInterrupt:
        print("\n✅ Deployment stopped by user.")

## 6. Additional Debugging Options

You can modify the visualization function above or add additional debugging code here. For example:

```python
# Save observations to disk
import json
from datetime import datetime

def save_observation_debug(observation, action, step, save_dir="./debug_observations"):
    import os
    os.makedirs(save_dir, exist_ok=True)
    
    # Save images
    for img_key in observation.keys():
        if img_key.startswith('observation.image'):
            img = observation[img_key].squeeze(0).cpu().numpy()
            if img.shape[0] in [1, 3]:
                img = img.transpose(1, 2, 0)
            Image.fromarray((img * 255).astype(np.uint8)).save(
                f"{save_dir}/step_{step:04d}_{img_key.replace('observation.image.', '')}.png"
            )
    
    # Save state and action
    debug_data = {
        'step': step,
        'timestamp': datetime.now().isoformat(),
        'state': observation['observation.state'].cpu().numpy().tolist(),
        'action': action.cpu().numpy().tolist()
    }
    
    with open(f"{save_dir}/step_{step:04d}_data.json", 'w') as f:
        json.dump(debug_data, f, indent=2)
```

You can then call `save_observation_debug(observation, action, step)` in the deployment loop.